# **Spotify | Spotipy**

Spotify has an API that allows users to gather information about songs and even interact with other users and playlists. To make their usage in Python easier, someone created `spotipy`, a library with some convenient functions to send requests and collect data.

Create / log into a Spotify account (https://developer.spotify.com/dashboard/login) and follow these steps (only the "Register your App" section): https://developer.spotify.com/documentation/general/guides/authorization/app-settings/

In [1]:
# https://spotipy.readthedocs.io/en/2.19.0/

# Install spotipy
# As google colab starts each session like a new computer, we must install this each time.
# If you are working on your local machine, you only need to install it once.
!pip install spotipy --upgrade
!pip install urllib3 --upgrade 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.5/238.5 KB 23.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# import libraries
import pandas as pd
import requests
import json

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
# Initialize SpotiPy with user credentials
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
    client_id="f0c14e779fc44359804f713d435cb87b",
    client_secret="5f81b3d52ff34df0881895ab3d34edea"))

#### Searching songs with 'queries' with `sp.search`

This method allows you to find songs using Spotify's search engine. That's convenient when you don't have the exact "id" of a song.

In [4]:
# Search for 'Coldplay', restricted to the first 10 results.

results = sp.search(q="Coldplay", limit = 10)

Explore the object returned by the request. As it's a dictionary (with nested dictionaries inside), using `.keys` is a great way to see what's in there:

In [5]:
results.keys()

dict_keys(['tracks'])

In [6]:
# Only one key makes it seem like there'll be many subdictionaries, let's delve deeper.
results["tracks"].keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

This is the name of the first song returned by the API:

In [7]:
results["tracks"]["items"][0]["name"]

'Sparks'

As one song can have many artists, the artists are returned as a list: note the square brackets.

In [8]:
results["tracks"]["items"][0]["artists"]

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/4gzpq5DPGxSnKTe4SA8HAU'},
  'href': 'https://api.spotify.com/v1/artists/4gzpq5DPGxSnKTe4SA8HAU',
  'id': '4gzpq5DPGxSnKTe4SA8HAU',
  'name': 'Coldplay',
  'type': 'artist',
  'uri': 'spotify:artist:4gzpq5DPGxSnKTe4SA8HAU'}]

There are some other interesting features contained in the search results:

In [9]:
# https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-tracks
# The popularity of the track. The value will be between 0 and 100, with 100 being the most popular.

results["tracks"]["items"][0]["popularity"]

81

This is how Spotify identifies individual songs: with a Uniform Resource Identifier  or `uri`. (The `id` and the `url` are also ways to identify each song uniquely).

In [10]:
results["tracks"]["items"][0]["uri"]

'spotify:track:7D0RhFcb3CrfPuTJ0obrod'

Here we look for 10 songs by the Red Hot Chili Peppers and store the `uri` of the songs and their names.

In [11]:
# Send request and store the response.
red_hot = sp.search(q="Red Hot Chili Peppers", limit=10)

# Initialize empty lists that we will fill with information from our loop.
list_of_uri = []
list_of_song_names = []

# Iterate through the "items" (the songs),
# and append the "uri" and the "name" to the lists we created.
for item in red_hot["tracks"]["items"]:
    list_of_uri.append(item["uri"])
    list_of_song_names.append(item["name"])

# Print results.
print(list_of_uri)
print("\n")
print(list_of_song_names)

['spotify:track:3d9DChrdc6BOeFsbrZ3Is0', 'spotify:track:3xJu5hrOU9OvFQSGLQiwQS', 'spotify:track:3ZOEytgrvLwQaqXreDs2Jx', 'spotify:track:64BbK9SFKH2jk86U3dGj2P', 'spotify:track:1G391cbiT3v3Cywg8T7DM1', 'spotify:track:4dzbGvxqQ1DsF6m6RUlPwg', 'spotify:track:48UPSzbZjgc449aqz8bxox', 'spotify:track:0Kojfmpnf0A2yC1zyv39Zx', 'spotify:track:0uppYCG86ajpV2hSR3dJJ0', 'spotify:track:2aibwv5hGXSgw7Yru8IYTO']


['Under the Bridge', 'Anthony Kiedis', "Can't Stop", 'Otherside', 'Scar Tissue', 'Pedigree', 'Californication', 'Buried Alive', 'Give It Away', 'Snow (Hey Oh)']


#### Searching multiple artists

Here we first create a list of artists we want to gather songs from. Then we iterate through them and append the results to a big list called `results`.

In [12]:
artists = ["Red Hot Chili Peppers", "SCARR", "Whitney Houston"]

In [13]:
results = []

for artist in artists:
    results.append(sp.search(q=artist, limit=10)) 

In [14]:
# Let's look at the second element in the results list.
results[1]

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=SCARR&type=track&offset=0&limit=10',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4fxd5Ee7UefO4CUXgwJ7IP'},
       'href': 'https://api.spotify.com/v1/artists/4fxd5Ee7UefO4CUXgwJ7IP',
       'id': '4fxd5Ee7UefO4CUXgwJ7IP',
       'name': 'Giveon',
       'type': 'artist',
       'uri': 'spotify:artist:4fxd5Ee7UefO4CUXgwJ7IP'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ',
   

We can iterate through the `results` list and get just the names of all the songs:

In [15]:
song_names = []

for result in results:
    for item in result["tracks"]["items"]:
        song_names.append(item["name"])

In [16]:
song_names

['Under the Bridge',
 'Anthony Kiedis',
 "Can't Stop",
 'Otherside',
 'Scar Tissue',
 'Pedigree',
 'Californication',
 'Buried Alive',
 'Give It Away',
 'Snow (Hey Oh)',
 'Scarred',
 'Emotionally Scarred',
 'SCARR',
 'Scarred From Love',
 "Scarred Baby's",
 'MJ (feat. Quezz Ruthless)',
 'SoIcyBoyz 2 (feat. Pooh Shiesty, Foogiano & Tay Keith)',
 'Scarred',
 'Backends',
 'Scarred',
 'I Wanna Dance with Somebody (Who Loves Me)',
 'I Will Always Love You',
 'Higher Love',
 'I Have Nothing',
 'How Will I Know',
 'Greatest Love of All',
 'Saving All My Love for You',
 'Higher Love',
 'How Will I Know',
 "I'm Your Baby Tonight"]

### Playlists

Using spotipy, we can both build and read spotify playlists. Today, we will only show you how to read information from a playlist. However, if you wish to build one, we strongly encourage you read the [documentation](https://spotipy.readthedocs.io/en/2.19.0/) and explore further.

In [17]:
my_playlist = sp.user_playlist_tracks(user="spotify", playlist_id="spotify:playlist:0ce6Rmxf7QXroqa1wzjWY8")

Extract songs ID from a playlist

In [18]:
my_playlist

{'href': 'https://api.spotify.com/v1/playlists/0ce6Rmxf7QXroqa1wzjWY8/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2021-09-22T07:34:05Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/gperdigo'},
    'href': 'https://api.spotify.com/v1/users/gperdigo',
    'id': 'gperdigo',
    'type': 'user',
    'uri': 'spotify:user:gperdigo'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/579T5fNgfbDetyamUTAetw'},
       'href': 'https://api.spotify.com/v1/artists/579T5fNgfbDetyamUTAetw',
       'id': '579T5fNgfbDetyamUTAetw',
       'name': 'S+C+A+R+R',
       'type': 'artist',
       'uri': 'spotify:artist:579T5fNgfbDetyamUTAetw'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
    

In [19]:
my_playlist["items"][0]["track"]["uri"]

'spotify:track:5Tnx4R7Gwj1LZsfssfzchh'

### Audio features

You can check an explanation of the audio features [here](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/).

In [20]:
sp.audio_features("spotify:track:6Sy9BUbgFse0n0LPA5lwy5")

[{'danceability': 0.528,
  'energy': 0.965,
  'key': 11,
  'loudness': -7.984,
  'mode': 0,
  'speechiness': 0.0465,
  'acousticness': 0.141,
  'instrumentalness': 0.985,
  'liveness': 0.0797,
  'valence': 0.587,
  'tempo': 136.065,
  'type': 'audio_features',
  'id': '6Sy9BUbgFse0n0LPA5lwy5',
  'uri': 'spotify:track:6Sy9BUbgFse0n0LPA5lwy5',
  'track_href': 'https://api.spotify.com/v1/tracks/6Sy9BUbgFse0n0LPA5lwy5',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6Sy9BUbgFse0n0LPA5lwy5',
  'duration_ms': 225493,
  'time_signature': 4}]

### Creating a function that takes a song name and returns its audio features 

In [21]:
list_of_songs = []

def song_features(human_song_title):
    # Search for the song title you enter into the function, limited to the first 10 results.
    results = sp.search(q=human_song_title, limit = 10)['tracks']['items']
    
    # Create a loop, so we only select the parts of the json we need.
    for i in results:

        # Empty dictionary to be filled with the information below.
        track_dict = {}

        # Add the key artist and a corresponding value to the dictionary.
        track_dict['Artist'] = i['artists'][0]['name']
        # Add the key title and the corresponding value to the dictionary.
        track_dict['Title'] = i['name'] 
        # Add the key album and the corresponding value to the dictionary.
        track_dict['Album'] = i['album']['name']
        # Add the key audio description and the corresponding value to the dictionary.
        track_dict['Audio Description'] = sp.audio_features(i['id'])
        
        # Add the dictionary to the list list_of_songs.
        list_of_songs.append(track_dict)
    # Output list_of_songs.
    return list_of_songs

# Call the function with a song to test.
song_features("Under the Bridge")

[{'Artist': 'Red Hot Chili Peppers',
  'Title': 'Under the Bridge',
  'Album': 'Blood Sugar Sex Magik (Deluxe Edition)',
  'Audio Description': [{'danceability': 0.559,
    'energy': 0.345,
    'key': 4,
    'loudness': -13.496,
    'mode': 1,
    'speechiness': 0.0459,
    'acousticness': 0.0576,
    'instrumentalness': 0.000105,
    'liveness': 0.141,
    'valence': 0.458,
    'tempo': 84.581,
    'type': 'audio_features',
    'id': '3d9DChrdc6BOeFsbrZ3Is0',
    'uri': 'spotify:track:3d9DChrdc6BOeFsbrZ3Is0',
    'track_href': 'https://api.spotify.com/v1/tracks/3d9DChrdc6BOeFsbrZ3Is0',
    'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3d9DChrdc6BOeFsbrZ3Is0',
    'duration_ms': 264307,
    'time_signature': 4}]},
 {'Artist': 'owlh',
  'Title': 'Under the Bridge',
  'Album': 'cover to cover',
  'Audio Description': [{'danceability': 0.781,
    'energy': 0.355,
    'key': 9,
    'loudness': -8.158,
    'mode': 1,
    'speechiness': 0.0376,
    'acousticness': 0.471,
    'in

In [22]:
# Make a dataframe from the list of songs created in the function above.
df = pd.DataFrame(list_of_songs)

df

,Artist,Title,Album,Audio Description
0,Red Hot Chili Peppers,Under the Bridge,Blood Sugar Sex Magik (Deluxe Edition),"[{'danceability': 0.559, 'energy': 0.345, 'key..."
1,owlh,Under the Bridge,cover to cover,"[{'danceability': 0.781, 'energy': 0.355, 'key..."
2,Sam Hunt,Water Under The Bridge,Water Under The Bridge,"[{'danceability': 0.635, 'energy': 0.866, 'key..."
3,Red Hot Chili Peppers,Under the Bridge,Greatest Hits,"[{'danceability': 0.554, 'energy': 0.49, 'key'..."
4,Adele,Water Under the Bridge,25,"[{'danceability': 0.59, 'energy': 0.833, 'key'..."
5,Rockabye Baby!,Under the Bridge,Lullaby Renditions of Red Hot Chili Peppers,"[{'danceability': 0.704, 'energy': 0.06, 'key'..."
6,Red Hot Chili Peppers,Under the Bridge,Classic Rock Classic Hits,"[{'danceability': 0.559, 'energy': 0.345, 'key..."
7,Royal Philharmonic Orchestra,Under the Bridge,Classic Rock (75 Orchestral Rock Anthems),"[{'danceability': 0.452, 'energy': 0.282, 'key..."
8,Jack Harlow,Churchill Downs (feat. Drake),Come Home The Kids Miss You,"[{'danceability': 0.71, 'energy': 0.522, 'key'..."
9,Santana,Under the Bridge (feat. Andy Vargas),Guitar Heaven: The Greatest Guitar Classics Of...,"[{'danceability': 0.54, 'energy': 0.844, 'key'..."


As you can see, this DataFrame looks a bit off as the audio descriptions aren't expanded - all of the data is clumped together in one cell. Let's correct this, so we can see each audio feature as an individual column.

In [23]:
# Quick function we can use to select only the first item.
# This can also be done simply with [0], but we wanted to show you how you can incorporate a custom function into your work.

def first_value (x):
    return x[0]

# Making a DataFrame from the audio features of the songs in list_of_songs.
df_audio_features = pd.json_normalize(df['Audio Description'].apply(first_value))

df_audio_features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.559,0.345,4,-13.496,1,0.0459,0.0576,0.000105,0.1410,0.458,84.581,audio_features,3d9DChrdc6BOeFsbrZ3Is0,spotify:track:3d9DChrdc6BOeFsbrZ3Is0,https://api.spotify.com/v1/tracks/3d9DChrdc6BO...,https://api.spotify.com/v1/audio-analysis/3d9D...,264307,4
1,0.781,0.355,9,-8.158,1,0.0376,0.4710,0.878000,0.1250,0.363,131.978,audio_features,1AyRpRvLwUwkXwGGuNZHmN,spotify:track:1AyRpRvLwUwkXwGGuNZHmN,https://api.spotify.com/v1/tracks/1AyRpRvLwUwk...,https://api.spotify.com/v1/audio-analysis/1AyR...,244196,4
2,0.635,0.866,5,-2.695,1,0.0473,0.0697,0.000000,0.0955,0.853,133.889,audio_features,4viFAHmivkYQKkwLvwKOgg,spotify:track:4viFAHmivkYQKkwLvwKOgg,https://api.spotify.com/v1/tracks/4viFAHmivkYQ...,https://api.spotify.com/v1/audio-analysis/4viF...,169760,4
3,0.554,0.490,4,-8.046,1,0.0457,0.0168,0.000534,0.1360,0.513,84.275,audio_features,23NPGXlSaIqWzvxIRhM2oG,spotify:track:23NPGXlSaIqWzvxIRhM2oG,https://api.spotify.com/v1/tracks/23NPGXlSaIqW...,https://api.spotify.com/v1/audio-analysis/23NP...,265507,4
4,0.590,0.833,5,-6.503,0,0.0615,0.0142,0.000005,0.1050,0.538,94.963,audio_features,4jL6WWKFDqCOPo2hC3VhSS,spotify:track:4jL6WWKFDqCOPo2hC3VhSS,https://api.spotify.com/v1/tracks/4jL6WWKFDqCO...,https://api.spotify.com/v1/audio-analysis/4jL6...,240439,4
5,0.704,0.060,4,-21.087,1,0.0907,0.9460,0.912000,0.1090,0.791,158.085,audio_features,45VCZtguiCNu5zJ3YkmcWC,spotify:track:45VCZtguiCNu5zJ3YkmcWC,https://api.spotify.com/v1/tracks/45VCZtguiCNu...,https://api.spotify.com/v1/audio-analysis/45VC...,259693,4
6,0.559,0.345,4,-13.495,1,0.0465,0.0571,0.000106,0.1410,0.466,84.581,audio_features,0U3EDeSJKOHfgTmBU6V3kq,spotify:track:0U3EDeSJKOHfgTmBU6V3kq,https://api.spotify.com/v1/tracks/0U3EDeSJKOHf...,https://api.spotify.com/v1/audio-analysis/0U3E...,264307,4
7,0.452,0.282,4,-14.303,1,0.0279,0.6710,0.888000,0.0968,0.257,84.035,audio_features,0oeUaI1nT5TNFaZdFPV33G,spotify:track:0oeUaI1nT5TNFaZdFPV33G,https://api.spotify.com/v1/tracks/0oeUaI1nT5TN...,https://api.spotify.com/v1/audio-analysis/0oeU...,222893,4
8,0.710,0.522,11,-9.102,1,0.3600,0.6150,0.000000,0.1100,0.370,96.448,audio_features,3EMp20j5E42MxfFbsEsIvD,spotify:track:3EMp20j5E42MxfFbsEsIvD,https://api.spotify.com/v1/tracks/3EMp20j5E42M...,https://api.spotify.com/v1/audio-analysis/3EMp...,309327,4
9,0.540,0.844,1,-4.986,0,0.0871,0.2710,0.000009,0.1550,0.644,87.518,audio_features,1AK5AYYFRCnd6qKrmh1N5t,spotify:track:1AK5AYYFRCnd6qKrmh1N5t,https://api.spotify.com/v1/tracks/1AK5AYYFRCnd...,https://api.spotify.com/v1/audio-analysis/1AK5...,309547,4


In [24]:
# Merge the expanded audio features with the original DataFrame.
new_df = pd.merge(df, df_audio_features, left_index=True, right_index=True)

# Drop the old ugly column where all the audio features are clumped together.
new_df.drop('Audio Description', axis=1, inplace=True)

new_df

,Artist,Title,Album,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Red Hot Chili Peppers,Under the Bridge,Blood Sugar Sex Magik (Deluxe Edition),0.559,0.345,4,-13.496,1,0.0459,0.0576,...,0.1410,0.458,84.581,audio_features,3d9DChrdc6BOeFsbrZ3Is0,spotify:track:3d9DChrdc6BOeFsbrZ3Is0,https://api.spotify.com/v1/tracks/3d9DChrdc6BO...,https://api.spotify.com/v1/audio-analysis/3d9D...,264307,4
1,owlh,Under the Bridge,cover to cover,0.781,0.355,9,-8.158,1,0.0376,0.4710,...,0.1250,0.363,131.978,audio_features,1AyRpRvLwUwkXwGGuNZHmN,spotify:track:1AyRpRvLwUwkXwGGuNZHmN,https://api.spotify.com/v1/tracks/1AyRpRvLwUwk...,https://api.spotify.com/v1/audio-analysis/1AyR...,244196,4
2,Sam Hunt,Water Under The Bridge,Water Under The Bridge,0.635,0.866,5,-2.695,1,0.0473,0.0697,...,0.0955,0.853,133.889,audio_features,4viFAHmivkYQKkwLvwKOgg,spotify:track:4viFAHmivkYQKkwLvwKOgg,https://api.spotify.com/v1/tracks/4viFAHmivkYQ...,https://api.spotify.com/v1/audio-analysis/4viF...,169760,4
3,Red Hot Chili Peppers,Under the Bridge,Greatest Hits,0.554,0.490,4,-8.046,1,0.0457,0.0168,...,0.1360,0.513,84.275,audio_features,23NPGXlSaIqWzvxIRhM2oG,spotify:track:23NPGXlSaIqWzvxIRhM2oG,https://api.spotify.com/v1/tracks/23NPGXlSaIqW...,https://api.spotify.com/v1/audio-analysis/23NP...,265507,4
4,Adele,Water Under the Bridge,25,0.590,0.833,5,-6.503,0,0.0615,0.0142,...,0.1050,0.538,94.963,audio_features,4jL6WWKFDqCOPo2hC3VhSS,spotify:track:4jL6WWKFDqCOPo2hC3VhSS,https://api.spotify.com/v1/tracks/4jL6WWKFDqCO...,https://api.spotify.com/v1/audio-analysis/4jL6...,240439,4
5,Rockabye Baby!,Under the Bridge,Lullaby Renditions of Red Hot Chili Peppers,0.704,0.060,4,-21.087,1,0.0907,0.9460,...,0.1090,0.791,158.085,audio_features,45VCZtguiCNu5zJ3YkmcWC,spotify:track:45VCZtguiCNu5zJ3YkmcWC,https://api.spotify.com/v1/tracks/45VCZtguiCNu...,https://api.spotify.com/v1/audio-analysis/45VC...,259693,4
6,Red Hot Chili Peppers,Under the Bridge,Classic Rock Classic Hits,0.559,0.345,4,-13.495,1,0.0465,0.0571,...,0.1410,0.466,84.581,audio_features,0U3EDeSJKOHfgTmBU6V3kq,spotify:track:0U3EDeSJKOHfgTmBU6V3kq,https://api.spotify.com/v1/tracks/0U3EDeSJKOHf...,https://api.spotify.com/v1/audio-analysis/0U3E...,264307,4
7,Royal Philharmonic Orchestra,Under the Bridge,Classic Rock (75 Orchestral Rock Anthems),0.452,0.282,4,-14.303,1,0.0279,0.6710,...,0.0968,0.257,84.035,audio_features,0oeUaI1nT5TNFaZdFPV33G,spotify:track:0oeUaI1nT5TNFaZdFPV33G,https://api.spotify.com/v1/tracks/0oeUaI1nT5TN...,https://api.spotify.com/v1/audio-analysis/0oeU...,222893,4
8,Jack Harlow,Churchill Downs (feat. Drake),Come Home The Kids Miss You,0.710,0.522,11,-9.102,1,0.3600,0.6150,...,0.1100,0.370,96.448,audio_features,3EMp20j5E42MxfFbsEsIvD,spotify:track:3EMp20j5E42MxfFbsEsIvD,https://api.spotify.com/v1/tracks/3EMp20j5E42M...,https://api.spotify.com/v1/audio-analysis/3EMp...,309327,4
9,Santana,Under the Bridge (feat. Andy Vargas),Guitar Heaven: The Greatest Guitar Classics Of...,0.540,0.844,1,-4.986,0,0.0871,0.2710,...,0.1550,0.644,87.518,audio_features,1AK5AYYFRCnd6qKrmh1N5t,spotify:track:1AK5AYYFRCnd6qKrmh1N5t,https://api.spotify.com/v1/tracks/1AK5AYYFRCnd...,https://api.spotify.com/v1/audio-analysis/1AK5...,309547,4



### Collect a big dataframe of songs with their audio features

- Start by looking for a playlist on spotify (it does not have to be your playlist), and copy its url.

- Extract the audio features for each song on your playlist.

- Collect the link of many playlists and do the same for all of them.

- Structure the information as a dataframe where each row is a song and the columns are audio features.

In [25]:
list_of_playlists = ["spotify:playlist:2zjepkjZxLpeIBlvPCWIHl",
                    "spotify:playlist:0ce6Rmxf7QXroqa1wzjWY8"]
track_list = []
for i in list_of_playlists:
    individual_playlist = sp.user_playlist_tracks(user="spotify", playlist_id=i)['items']
    for j in individual_playlist:
        track_dict = {}
        track_dict['Artist'] = j['track']['artists'][0]['name']
        track_dict['Title'] = j['track']['name']
        track_dict['Album'] = j['track']['album']['name']
        track_dict['Audio Description'] = sp.audio_features(j['track']['id'])
        track_list.append(track_dict)

print(track_list)

[{'Artist': 'James Blake', 'Title': 'Words That We Both Know', 'Album': '200 Press EP', 'Audio Description': [{'danceability': 0.5, 'energy': 0.439, 'key': 7, 'loudness': -9.207, 'mode': 0, 'speechiness': 0.107, 'acousticness': 0.927, 'instrumentalness': 0.000445, 'liveness': 0.281, 'valence': 0.42, 'tempo': 62.058, 'type': 'audio_features', 'id': '4BZtAKMwi4aM18Zr2mTIgt', 'uri': 'spotify:track:4BZtAKMwi4aM18Zr2mTIgt', 'track_href': 'https://api.spotify.com/v1/tracks/4BZtAKMwi4aM18Zr2mTIgt', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4BZtAKMwi4aM18Zr2mTIgt', 'duration_ms': 63025, 'time_signature': 5}]}, {'Artist': 'Gene Wilder', 'Title': 'Pure Imagination - From "Willy Wonka & The Chocolate Factory" Soundtrack', 'Album': 'Willy Wonka & The Chocolate Factory', 'Audio Description': [{'danceability': 0.251, 'energy': 0.115, 'key': 10, 'loudness': -20.223, 'mode': 0, 'speechiness': 0.038, 'acousticness': 0.695, 'instrumentalness': 2.11e-05, 'liveness': 0.617, 'valence': 0.1

In [26]:
playlist_df = pd.DataFrame(track_list)

In [27]:
df_a_f = pd.json_normalize(playlist_df['Audio Description'].apply(first_value))

In [28]:
new_playlist_df = pd.merge(playlist_df, df_a_f, left_index=True, right_index=True)

In [29]:
new_playlist_df.drop('Audio Description', axis=1, inplace=True)

In [30]:
new_playlist_df

,Artist,Title,Album,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,James Blake,Words That We Both Know,200 Press EP,0.500,0.439,7,-9.207,0,0.1070,0.92700,...,0.2810,0.4200,62.058,audio_features,4BZtAKMwi4aM18Zr2mTIgt,spotify:track:4BZtAKMwi4aM18Zr2mTIgt,https://api.spotify.com/v1/tracks/4BZtAKMwi4aM...,https://api.spotify.com/v1/audio-analysis/4BZt...,63025,5
1,Gene Wilder,"Pure Imagination - From ""Willy Wonka & The Cho...",Willy Wonka & The Chocolate Factory,0.251,0.115,10,-20.223,0,0.0380,0.69500,...,0.6170,0.1160,70.645,audio_features,4f4HLo6L9CJNhD4mEYxrCL,spotify:track:4f4HLo6L9CJNhD4mEYxrCL,https://api.spotify.com/v1/tracks/4f4HLo6L9CJN...,https://api.spotify.com/v1/audio-analysis/4f4H...,260227,4
2,Bob Dorough,Polka Dots and Moonbeams,Devil May Care,0.580,0.133,0,-18.177,0,0.0962,0.79500,...,0.1800,0.2110,57.658,audio_features,2wy2ScXvlpguDeo4NnwvOm,spotify:track:2wy2ScXvlpguDeo4NnwvOm,https://api.spotify.com/v1/tracks/2wy2ScXvlpgu...,https://api.spotify.com/v1/audio-analysis/2wy2...,205360,3
3,Isaiah Rashad,R.I.P. Kevin Miller,Cilvia Demo,0.703,0.835,8,-4.456,0,0.2700,0.22700,...,0.1350,0.6020,136.158,audio_features,6zSwnPvoqQ2bzvYMlt3u4u,spotify:track:6zSwnPvoqQ2bzvYMlt3u4u,https://api.spotify.com/v1/tracks/6zSwnPvoqQ2b...,https://api.spotify.com/v1/audio-analysis/6zSw...,229219,4
4,Jon Brion,"Phone Call - From ""Eternal Sunshine of the Spo...",Eternal Sunshine Of The Spotless Mind,0.498,0.377,1,-19.858,1,0.0474,0.98200,...,0.1070,0.5450,126.252,audio_features,7LYeytL8R2fuqnsW1s0k86,spotify:track:7LYeytL8R2fuqnsW1s0k86,https://api.spotify.com/v1/tracks/7LYeytL8R2fu...,https://api.spotify.com/v1/audio-analysis/7LYe...,62600,4
5,Action Bronson,Terry,Mr. Wonderful,0.376,0.620,1,-5.840,1,0.3670,0.70700,...,0.1070,0.0477,87.538,audio_features,6bj42Hzr3RErxhfgSTdSx6,spotify:track:6bj42Hzr3RErxhfgSTdSx6,https://api.spotify.com/v1/tracks/6bj42Hzr3REr...,https://api.spotify.com/v1/audio-analysis/6bj4...,289493,4
6,Benny Sings,Realize,Art,0.829,0.237,1,-9.388,1,0.1130,0.48200,...,0.0987,0.8070,75.512,audio_features,6HKq2PtG1X7f5QaVBlGTGr,spotify:track:6HKq2PtG1X7f5QaVBlGTGr,https://api.spotify.com/v1/tracks/6HKq2PtG1X7f...,https://api.spotify.com/v1/audio-analysis/6HKq...,185893,4
7,Oneohtrix Point Never,Sleep Dealer,Replica,0.587,0.782,10,-9.733,0,0.0837,0.64000,...,0.6080,0.2730,112.159,audio_features,5KXSZjxPDUHJUGfVExhOEh,spotify:track:5KXSZjxPDUHJUGfVExhOEh,https://api.spotify.com/v1/tracks/5KXSZjxPDUHJ...,https://api.spotify.com/v1/audio-analysis/5KXS...,190417,4
8,Pink Floyd,Welcome to the Machine,Wish You Were Here,0.381,0.358,0,-12.423,1,0.0365,0.52400,...,0.0747,0.0360,133.271,audio_features,5VWC7v2dC2K0SIIjT9WTLN,spotify:track:5VWC7v2dC2K0SIIjT9WTLN,https://api.spotify.com/v1/tracks/5VWC7v2dC2K0...,https://api.spotify.com/v1/audio-analysis/5VWC...,451681,4
9,Boards of Canada,Dawn Chorus,Geogaddi,0.418,0.686,5,-8.658,1,0.0350,0.00177,...,0.8630,0.5470,75.241,audio_features,7LY0R33pPIFr0Y64VZC9Wo,spotify:track:7LY0R33pPIFr0Y64VZC9Wo,https://api.spotify.com/v1/tracks/7LY0R33pPIFr...,https://api.spotify.com/v1/audio-analysis/7LY0...,235493,1
